In [1]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf, explode, count
from pyspark.sql.types import IntegerType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
19,application_1547576614992_0023,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
sentenceDataFrame = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Spark could use Spark classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

In [5]:
sentenceDataFrame.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Spark coul...|
|  2|Logistic,regressi...|
+---+--------------------+

In [4]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [24]:
tokenized = tokenizer.transform(sentenceDataFrame)
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Spark coul...|[i, wish, spark, ...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+

In [25]:
tokenized.printSchema()

root
 |-- id: long (nullable = true)
 |-- sentence: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)

In [48]:
counts_df = tokenized\
    .withColumn('word', explode(col('words')))\
    .groupBy('word')\
    .agg(count("*").alias('word_count'))\
    .orderBy(col('word_count').desc())

In [49]:
counts_df.show()

+--------------------+----------+
|                word|word_count|
+--------------------+----------+
|               spark|         3|
|                   i|         2|
|             classes|         1|
|               could|         1|
|                 use|         1|
|logistic,regressi...|         1|
|                wish|         1|
|               heard|         1|
|                  hi|         1|
|               about|         1|
+--------------------+----------+